In [ ]:
!huggingface-cli login

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio

In [ ]:
!pip install peft

In [ ]:
!pip install -q datasets

In [ ]:
!pip install langchain faiss-cpu sentence-transformers transformers


In [ ]:
!pip install PyPDF2

In [ ]:
!pip install -U langchain-community

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

vectorstore = FAISS.load_local(
    "/content/drive/MyDrive/rag_index",
    embeddings=embedding_model,
    allow_dangerous_deserialization=True
)


# Data structure and Memory System

In [ ]:
# ========================================================================
# CLEAN & SIMPLE WELLBEING LLM SYSTEM
# ========================================================================

import torch
import json
import os
from datetime import datetime # Import datetime
from typing import Dict, List
import gradio as gr

# ========================================================================
# 1. SIMPLE DATA STORAGE
# ========================================================================

class SimpleDataManager:
    """Simple file-based storage for user data and recommendations"""

    def __init__(self, data_dir="wellbeing_data"):
        self.data_dir = data_dir
        os.makedirs(data_dir, exist_ok=True)
        os.makedirs(f"{data_dir}/users", exist_ok=True)
        os.makedirs(f"{self.data_dir}/recommendations", exist_ok=True) # Corrected path

    def save_user_week(self, user_id: str, week_data: Dict):
        """Save weekly data for a user"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            # Load existing data
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
            else:
                user_data = {"user_id": user_id, "weeks": []}

            # Add new week
            user_data["weeks"].append(week_data)

            # Keep only last 8 weeks
            user_data["weeks"] = user_data["weeks"][-8:]

            # Save
            with open(user_file, 'w') as f:
                json.dump(user_data, f, indent=2)

            print(f"✅ Saved week data for user {user_id}")

        except Exception as e:
            print(f"❌ Error saving user data: {str(e)}")

    def get_user_history(self, user_id: str) -> List[Dict]:
        """Get user's weekly history"""
        user_file = f"{self.data_dir}/users/{user_id}.json"

        try:
            if os.path.exists(user_file):
                with open(user_file, 'r') as f:
                    user_data = json.load(f)
                    history = user_data.get("weeks", [])
                    print(f"📊 Retrieved {len(history)} weeks of history for {user_id}")
                    return history
        except Exception as e:
            print(f"❌ Error loading user history: {str(e)}")

        return []

    def save_recommendation(self, user_id: str, week_start: str, recommendation: str):
        """Save LLM recommendation"""
        # Clean week_start for filename (remove invalid characters)
        clean_week = week_start.replace("/", "-").replace(":", "-")
        rec_file = f"{self.data_dir}/recommendations/{user_id}_{clean_week}.json"

        try:
            rec_data = {
                "user_id": user_id,
                "week_start": week_start,
                "recommendation": recommendation,
                "timestamp": datetime.now().isoformat()
            }

            with open(rec_file, 'w') as f:
                json.dump(rec_data, f, indent=2)

            print(f"✅ Saved recommendation for user {user_id}, week {week_start}")

        except Exception as e:
            print(f"❌ Error saving recommendation: {str(e)}")

    def get_last_recommendation(self, user_id: str) -> str:
        """Get user's last recommendation"""
        rec_dir = f"{self.data_dir}/recommendations"

        try:
            # Check if recommendations directory exists
            if not os.path.exists(rec_dir):
                return ""

            # Find latest recommendation file for this user
            user_files = [f for f in os.listdir(rec_dir) if f.startswith(f"{user_id}_") and f.endswith('.json')]

            if user_files:
                # Sort by date and get latest
                user_files.sort(reverse=True)
                latest_file = f"{rec_dir}/{user_files[0]}"

                with open(latest_file, 'r') as f:
                    rec_data = json.load(f)
                    recommendation = rec_data.get("recommendation", "")
                    print(f"📝 Retrieved last recommendation for {user_id} ({len(recommendation)} chars)")
                    return recommendation

        except Exception as e:
            print(f"❌ Error loading last recommendation: {str(e)}")

        return ""

    def get_user_stats(self, user_id: str) -> Dict:
        """Get basic stats about user's data (bonus method)"""
        history = self.get_user_history(user_id)

        if not history:
            return {"total_weeks": 0}

        return {
            "total_weeks": len(history),
            "first_week": history[0].get("week_start", "Unknown"),
            "latest_week": history[-1].get("week_start", "Unknown"),
            "avg_steps": sum(week.get("total_steps", 0) for week in history) // len(history),
            "avg_sleep": sum(week.get("avg_sleep", 0) for week in history) / len(history)
        }

# ========================================================================
# 2. SIMPLE LLM SYSTEM
# ========================================================================

import torch
import re
from typing import Dict, List

class WellbeingLLM:
    """Improved LLM system for wellbeing recommendations"""

    def __init__(self, base_model_id: str, adapter_path: str):
        self.model = None
        self.tokenizer = None
        self.device = None
        self.load_model(base_model_id, adapter_path)

    def load_model(self, base_model_id: str, adapter_path: str):
        """Load the LLM model with better error handling"""
        try:
            print("🤖 Loading LLM...")

            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
            from peft import PeftModel

            # Clear GPU memory and check device
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                print(f"🔍 GPU available: {torch.cuda.get_device_name()}")
            else:
                print("🔍 Using CPU")

            # Load tokenizer
            print("📝 Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer loaded")

            # Quantization config
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float16,
            )
            print("⚙️ Quantization config ready")

            # Load base model
            print("🧠 Loading base model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model_id,
                device_map="auto",
                torch_dtype=torch.float16,
                quantization_config=bnb_config,
                trust_remote_code=True
            )
            print("✅ Base model loaded")

            # Load fine-tuned adapter
            print("🎯 Loading fine-tuned adapter...")
            self.model = PeftModel.from_pretrained(self.model, adapter_path)
            self.model.eval()

            # Store device for later use
            self.device = next(self.model.parameters()).device
            print(f"✅ LLM loaded successfully on {self.device}")

        except Exception as e:
            print(f"❌ LLM loading failed: {str(e)}")
            import traceback
            traceback.print_exc()
            self.model = None
            self.tokenizer = None
            self.device = None

    def generate_recommendation(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
        """Generate personalized recommendation with improved prompt"""

        if not self.model or not self.tokenizer:
            print("❌ No model loaded")
            return "Model not available. Please check model loading."

        try:
            # Build better structured prompt
            prompt = self._build_structured_prompt(current_week, user_history, last_recommendation)

            print(f"🔍 Prompt ready ({len(prompt)} chars)")
            print(f"🔍 Preview: {prompt[:150]}...")

            # Tokenize with better parameters
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=1536,  # Increased for more context
                padding=False
            )

            # Move to device
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Generate with improved parameters
            print("🔍 Generating response...")
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=400,      # More tokens for detailed response
                    min_new_tokens=100,      # Ensure substantial response
                    temperature=0.8,         # Higher for more creativity
                    top_p=0.92,             # Allow diverse responses
                    top_k=50,               # Add top-k sampling
                    do_sample=True,
                    repetition_penalty=1.1,
                    no_repeat_ngram_size=3,  # Prevent repetitive phrases
                    pad_token_id=self.tokenizer.eos_token_id,
                    early_stopping=False     # Let it generate full response
                )

            # Extract and clean response
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = full_response[len(prompt):].strip()

            # Clean response
            response = self._clean_response(response)

            print(f"✅ Generated response ({len(response)} chars)")
            print(f"🔍 Response preview: {response[:150]}...")

            # Validate personalization
            is_personalized = self._validate_personalization(response, current_week)

            if len(response) < 50:
                print("⚠️ Response too short")
                return f"Generated response was too short: '{response}'"

            if not is_personalized:
                print("⚠️ Response may not be personalized")

            return response

        except Exception as e:
            print(f"❌ Generation error: {str(e)}")
            import traceback
            traceback.print_exc()
            return f"Generation failed: {str(e)}"

    def _build_structured_prompt(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
        """Build a well-structured prompt for the LLM"""

        # Extract key metrics
        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)
        avg_mood = current_week.get('avg_mood', 0)
        exercise_sessions = current_week.get('exercise_sessions', 0)

        # Handle food data with units
        food_data = current_week.get('food_data', {})
        food_summary = self._format_food_data(food_data)

        # Build progress context
        progress_context = self._build_progress_context(current_week, user_history)

        # Create structured prompt
        prompt = f"""As a health and wellness expert, analyze this person's weekly health data and provide specific, personalized recommendations.

CURRENT WEEK DATA:
• Steps taken: {total_steps:,} (target: 70,000/week)
• Cardio zone minutes: {zone_minutes} (target: 150/week)
• Exercise sessions: {exercise_sessions}
• Sleep average: {avg_sleep:.1f} hours/night (target: 7-8 hours)
• Mood average: {avg_mood:.1f}/10

NUTRITION THIS WEEK:
{food_summary}

{progress_context}

Based on this person's specific numbers and patterns, provide personalized recommendations that reference their actual data. Focus on areas that need improvement and acknowledge what they're doing well.

Personalized recommendations:"""

        return prompt

    def _format_food_data(self, food_data: Dict) -> str:
        """Format food data with proper units"""
        if not food_data:
            return "• No food data provided"

        food_lines = []
        unit_mapping = {
            'dairy_liters': ('Dairy', 'L'),
            'water_liters': ('Water', 'L'),
            'legumes_grams': ('Legumes', 'g'),
            'meat_grams': ('Meat', 'g'),
            'fruits_grams': ('Fruits', 'g'),
            'vegetables_grams': ('Vegetables', 'g'),
            'grains_grams': ('Grains', 'g'),
            'nuts_seeds_grams': ('Nuts/Seeds', 'g')
        }

        for key, amount in food_data.items():
            if amount > 0 and key in unit_mapping:
                name, unit = unit_mapping[key]
                food_lines.append(f"• {name}: {amount:.0f}{unit}")
            elif amount > 0:
                # Handle old format
                clean_name = key.replace('_', ' ').title()
                food_lines.append(f"• {clean_name}: {amount}")

        return '\n'.join(food_lines) if food_lines else "• No significant food intake recorded"

    def _build_progress_context(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Build progress context from user history"""
        if len(user_history) < 2:
            return "PROGRESS CONTEXT:\n• First week or limited history available"

        prev_week = user_history[-2]

        # Calculate changes
        step_change = current_week.get('total_steps', 0) - prev_week.get('total_steps', 0)
        zone_change = current_week.get('zone_minutes', 0) - prev_week.get('zone_minutes', 0)
        sleep_change = current_week.get('avg_sleep', 0) - prev_week.get('avg_sleep', 0)

        context = f"""PROGRESS FROM LAST WEEK:
• Steps: {step_change:+,} change
• Zone minutes: {zone_change:+} change
• Sleep: {sleep_change:+.1f} hours change
• Total weeks tracked: {len(user_history)}"""

        return context

    def _clean_response(self, response: str) -> str:
        """Clean the generated response"""
        # Remove common unwanted patterns
        response = re.sub(r'^(Response:|Recommendations?:)', '', response, flags=re.IGNORECASE)
        response = response.strip()

        # Remove excessive newlines
        response = re.sub(r'\n{3,}', '\n\n', response)

        return response

    def _validate_personalization(self, response: str, current_week: Dict) -> bool:
        """Check if response mentions user's specific data"""
        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)

        # Check if response mentions actual user data
        mentions_steps = str(total_steps) in response or f"{total_steps:,}" in response
        mentions_zone = str(zone_minutes) in response
        mentions_sleep = f"{avg_sleep:.1f}" in response or str(int(avg_sleep)) in response

        personalized = mentions_steps or mentions_zone or mentions_sleep

        print(f"🔍 Personalization check:")
        print(f"   Steps ({total_steps:,}): {mentions_steps}")
        print(f"   Zone mins ({zone_minutes}): {mentions_zone}")
        print(f"   Sleep ({avg_sleep:.1f}h): {mentions_sleep}")
        print(f"   Overall personalized: {personalized}")

        return personalized

    def test_model(self) -> str:
        """Test if model is working with simple input"""
        if not self.model or not self.tokenizer:
            return "❌ Model not loaded"

        try:
            test_prompt = "Recommend healthy foods:"
            inputs = self.tokenizer(test_prompt, return_tensors="pt").to(self.device)

            with torch.no_grad():
                outputs = self.model.generate(**inputs, max_new_tokens=50, temperature=0.7)

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return f"✅ Model test successful: {response[len(test_prompt):].strip()}"

        except Exception as e:
            return f"❌ Model test failed: {str(e)}"

#Well being LLM

In [ ]:
# ========================================================================
# 2. SIMPLE LLM SYSTEM
# ========================================================================

import torch
import re
from typing import Dict, List

class WellbeingLLM:
    """Improved LLM system for wellbeing recommendations"""

    def __init__(self, base_model_id: str, adapter_path: str):
        self.model = None
        self.tokenizer = None
        self.device = None
        self.load_model(base_model_id, adapter_path)

    def load_model(self, base_model_id: str, adapter_path: str):
        """Load the LLM model with better error handling"""
        try:
            print("🤖 Loading LLM...")

            from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
            from peft import PeftModel

            # Clear GPU memory and check device
            if torch.cuda.is_available():
                torch.cuda.empty_cache()
                print(f"🔍 GPU available: {torch.cuda.get_device_name()}")
            else:
                print("🔍 Using CPU")

            # Load tokenizer
            print("📝 Loading tokenizer...")
            self.tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
            if self.tokenizer.pad_token is None:
                self.tokenizer.pad_token = self.tokenizer.eos_token
            print("✅ Tokenizer loaded")

            # Quantization config
            bnb_config = BitsAndBytesConfig(
                load_in_4bit=True,
                bnb_4bit_quant_type="nf4",
                bnb_4bit_use_double_quant=True,
                bnb_4bit_compute_dtype=torch.float16,
            )
            print("⚙️ Quantization config ready")

            # Load base model
            print("🧠 Loading base model...")
            self.model = AutoModelForCausalLM.from_pretrained(
                base_model_id,
                device_map="auto",
                torch_dtype=torch.float16,
                quantization_config=bnb_config,
                trust_remote_code=True
            )
            print("✅ Base model loaded")

            # Load fine-tuned adapter
            print("🎯 Loading fine-tuned adapter...")
            self.model = PeftModel.from_pretrained(self.model, adapter_path)
            self.model.eval()

            # Store device for later use
            self.device = next(self.model.parameters()).device
            print(f"✅ LLM loaded successfully on {self.device}")

        except Exception as e:
            print(f"❌ LLM loading failed: {str(e)}")
            import traceback
            traceback.print_exc()
            self.model = None
            self.tokenizer = None
            self.device = None

    def generate_recommendation(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
        """Generate personalized recommendation with improved prompt"""

        if not self.model or not self.tokenizer:
            print("❌ No model loaded")
            return "Model not available. Please check model loading."

        try:
            # Build better structured prompt
            prompt = self._build_structured_prompt(current_week, user_history, last_recommendation)

            print(f"🔍 Prompt ready ({len(prompt)} chars)")
            print(f"🔍 Preview: {prompt[:150]}...")

            # Tokenize with better parameters
            inputs = self.tokenizer(
                prompt,
                return_tensors="pt",
                truncation=True,
                max_length=1536,  # Increased for more context
                padding=False
            )

            # Move to device
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Generate with improved parameters
            print("🔍 Generating response...")
            with torch.no_grad():
                outputs = self.model.generate(
                    **inputs,
                    max_new_tokens=400,      # More tokens for detailed response
                    min_new_tokens=100,      # Ensure substantial response
                    temperature=0.8,         # Higher for more creativity
                    top_p=0.92,             # Allow diverse responses
                    top_k=50,               # Add top-k sampling
                    do_sample=True,
                    repetition_penalty=1.1,
                    no_repeat_ngram_size=3,  # Prevent repetitive phrases
                    pad_token_id=self.tokenizer.eos_token_id,
                    early_stopping=False     # Let it generate full response
                )

            # Extract and clean response
            full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            response = full_response[len(prompt):].strip()

            # Clean response
            response = self._clean_response(response)

            print(f"✅ Generated response ({len(response)} chars)")
            print(f"🔍 Response preview: {response[:150]}...")

            # Validate personalization
            is_personalized = self._validate_personalization(response, current_week)

            if len(response) < 50:
                print("⚠️ Response too short")
                return f"Generated response was too short: '{response}'"

            if not is_personalized:
                print("⚠️ Response may not be personalized")

            return response

        except Exception as e:
            print(f"❌ Generation error: {str(e)}")
            import traceback
            traceback.print_exc()
            return f"Generation failed: {str(e)}"

    def _build_structured_prompt(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
        """Build a well-structured prompt for the LLM"""

        # Extract key metrics
        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)
        avg_mood = current_week.get('avg_mood', 0)
        exercise_sessions = current_week.get('exercise_sessions', 0)

        # Handle food data with units
        food_data = current_week.get('food_data', {})
        food_summary = self._format_food_data(food_data)

        # Build progress context
        progress_context = self._build_progress_context(current_week, user_history)

        # Create structured prompt
        prompt = f"""As a health and wellness expert, analyze this person's weekly health data and provide specific, personalized recommendations.

CURRENT WEEK DATA:
• Steps taken: {total_steps:,} (target: 70,000/week)
• Cardio zone minutes: {zone_minutes} (target: 150/week)
• Exercise sessions: {exercise_sessions}
• Sleep average: {avg_sleep:.1f} hours/night (target: 7-8 hours)
• Mood average: {avg_mood:.1f}/10

NUTRITION THIS WEEK:
{food_summary}

{progress_context}

Based on this person's specific numbers and patterns, provide personalized recommendations that reference their actual data. Focus on areas that need improvement and acknowledge what they're doing well.

Personalized recommendations:"""

        return prompt

    def _format_food_data(self, food_data: Dict) -> str:
        """Format food data with proper units"""
        if not food_data:
            return "• No food data provided"

        food_lines = []
        unit_mapping = {
            'dairy_liters': ('Dairy', 'L'),
            'water_liters': ('Water', 'L'),
            'legumes_grams': ('Legumes', 'g'),
            'meat_grams': ('Meat', 'g'),
            'fruits_grams': ('Fruits', 'g'),
            'vegetables_grams': ('Vegetables', 'g'),
            'grains_grams': ('Grains', 'g'),
            'nuts_seeds_grams': ('Nuts/Seeds', 'g')
        }

        for key, amount in food_data.items():
            if amount > 0 and key in unit_mapping:
                name, unit = unit_mapping[key]
                food_lines.append(f"• {name}: {amount:.0f}{unit}")
            elif amount > 0:
                # Handle old format
                clean_name = key.replace('_', ' ').title()
                food_lines.append(f"• {clean_name}: {amount}")

        return '\n'.join(food_lines) if food_lines else "• No significant food intake recorded"

    def _build_progress_context(self, current_week: Dict, user_history: List[Dict]) -> str:
        """Build progress context from user history"""
        if len(user_history) < 2:
            return "PROGRESS CONTEXT:\n• First week or limited history available"

        prev_week = user_history[-2]

        # Calculate changes
        step_change = current_week.get('total_steps', 0) - prev_week.get('total_steps', 0)
        zone_change = current_week.get('zone_minutes', 0) - prev_week.get('zone_minutes', 0)
        sleep_change = current_week.get('avg_sleep', 0) - prev_week.get('avg_sleep', 0)

        context = f"""PROGRESS FROM LAST WEEK:
• Steps: {step_change:+,} change
• Zone minutes: {zone_change:+} change
• Sleep: {sleep_change:+.1f} hours change
• Total weeks tracked: {len(user_history)}"""

        return context

    def _clean_response(self, response: str) -> str:
        """Clean the generated response"""
        # Remove common unwanted patterns
        response = re.sub(r'^(Response:|Recommendations?:)', '', response, flags=re.IGNORECASE)
        response = response.strip()

        # Remove excessive newlines
        response = re.sub(r'\n{3,}', '\n\n', response)

        return response

    def _validate_personalization(self, response: str, current_week: Dict) -> bool:
        """Check if response mentions user's specific data"""
        total_steps = current_week.get('total_steps', 0)
        zone_minutes = current_week.get('zone_minutes', 0)
        avg_sleep = current_week.get('avg_sleep', 0)

        # Check if response mentions actual user data
        mentions_steps = str(total_steps) in response or f"{total_steps:,}" in response
        mentions_zone = str(zone_minutes) in response
        mentions_sleep = f"{avg_sleep:.1f}" in response or str(int(avg_sleep)) in response

        personalized = mentions_steps or mentions_zone or mentions_sleep

        print(f"🔍 Personalization check:")
        print(f"   Steps ({total_steps:,}): {mentions_steps}")
        print(f"   Zone mins ({zone_minutes}): {mentions_zone}")
        print(f"   Sleep ({avg_sleep:.1f}h): {mentions_sleep}")
        print(f"   Overall personalized: {personalized}")

        return personalized

    def test_model(self) -> str:
        """Test if model is working with simple input"""
        if not self.model or not self.tokenizer:
            return "❌ Model not loaded"

        try:
            test_prompt = "Recommend healthy foods:"
            inputs = self.tokenizer(test_prompt, return_tensors="pt").to(self.device)

            with torch.no_grad():
                outputs = self.model.generate(**inputs, max_new_tokens=50, temperature=0.7)

            response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return f"✅ Model test successful: {response[len(test_prompt):].strip()}"

        except Exception as e:
            return f"❌ Model test failed: {str(e)}"


      # Add this method to your WellbeingLLM class:

def _build_structured_prompt_with_preferences(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
    """Build prompt including dietary preferences and allergies"""

    # Extract key metrics
    total_steps = current_week.get('total_steps', 0)
    zone_minutes = current_week.get('zone_minutes', 0)
    avg_sleep = current_week.get('avg_sleep', 0)
    avg_mood = current_week.get('avg_mood', 0)
    exercise_sessions = current_week.get('exercise_sessions', 0)

    # Handle food data with units
    food_data = current_week.get('food_data', {})
    food_summary = self._format_food_data(food_data)

    # Build progress context
    progress_context = self._build_progress_context(current_week, user_history)

    # NEW: Handle preferences
    preferences = current_week.get('preferences', {})
    preferences_context = self._build_preferences_context(preferences)

    # Create enhanced prompt
    prompt = f"""As a health and wellness expert, analyze this person's weekly health data and provide specific, personalized recommendations that respect their dietary preferences and allergies.

CURRENT WEEK DATA:
• Steps taken: {total_steps:,} (target: 70,000/week)
• Cardio zone minutes: {zone_minutes} (target: 150/week)
• Exercise sessions: {exercise_sessions}
• Sleep average: {avg_sleep:.1f} hours/night (target: 7-8 hours)
• Mood average: {avg_mood:.1f}/10

NUTRITION THIS WEEK:
{food_summary}

{preferences_context}

{progress_context}

IMPORTANT SAFETY REQUIREMENTS:
- NEVER recommend foods that contain their allergens
- Respect their dietary restrictions completely
- Suggest alternatives that fit their preferences
- If unsure about allergens, mention checking labels

Based on this person's specific data, dietary needs, and preferences, provide personalized recommendations that are safe and appealing to them.

Personalized recommendations:"""

    return prompt

def _build_preferences_context(self, preferences: Dict) -> str:
    """Build dietary preferences context for the LLM"""
    if not preferences:
        return "DIETARY PREFERENCES:\n• No specific preferences provided"

    context_lines = ["DIETARY PREFERENCES & RESTRICTIONS:"]

    # Diet type
    diet_type = preferences.get('diet_type', 'No Preference')
    if diet_type != 'No Preference':
        context_lines.append(f"• Diet Type: {diet_type}")

    # CRITICAL: Allergies first
    allergies = preferences.get('allergies', [])
    other_allergies = preferences.get('other_allergies', '')

    if allergies or other_allergies:
        context_lines.append("• ⚠️ ALLERGIES/RESTRICTIONS (MUST AVOID):")
        for allergy in allergies:
            context_lines.append(f"  - {allergy}")
        if other_allergies:
            context_lines.append(f"  - {other_allergies}")

    # Cuisine preferences
    cuisine_prefs = preferences.get('cuisine_preferences', [])
    if cuisine_prefs:
        cuisine_list = ", ".join(cuisine_prefs)
        context_lines.append(f"• Enjoys: {cuisine_list} cuisines")

    # Meal preferences
    meal_prefs = preferences.get('meal_preferences', [])
    if meal_prefs:
        meal_list = ", ".join(meal_prefs)
        context_lines.append(f"• Meal style preferences: {meal_list}")

    return '\n'.join(context_lines)

# Update the main generate_recommendation method:
def generate_recommendation(self, current_week: Dict, user_history: List[Dict], last_recommendation: str) -> str:
    """Generate personalized recommendation with preferences support"""

    if not self.model or not self.tokenizer:
        print("❌ No model loaded")
        return "Model not available. Please check model loading."

    try:
        # Check if preferences are included
        has_preferences = 'preferences' in current_week

        if has_preferences:
            # Use enhanced prompt with preferences
            prompt = self._build_structured_prompt_with_preferences(current_week, user_history, last_recommendation)
            print("🍽️ Using enhanced prompt with dietary preferences")
        else:
            # Use original prompt
            prompt = self._build_structured_prompt(current_week, user_history, last_recommendation)
            print("📝 Using standard prompt")

        print(f"🔍 Prompt ready ({len(prompt)} chars)")

        # Show allergy safety check
        if has_preferences:
            allergies = current_week.get('preferences', {}).get('allergies', [])
            if allergies:
                print(f"⚠️ SAFETY: Will avoid recommending: {', '.join(allergies)}")

        # Tokenize with better parameters
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=1536,
            padding=False
        )

        # Move to device
        inputs = {k: v.to(self.device) for k, v in inputs.items()}

        # Generate with improved parameters
        print("🔍 Generating personalized response...")
        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=450,      # More tokens for detailed dietary advice
                min_new_tokens=120,      # Ensure substantial response
                temperature=0.8,
                top_p=0.92,
                top_k=50,
                do_sample=True,
                repetition_penalty=1.1,
                no_repeat_ngram_size=3,
                pad_token_id=self.tokenizer.eos_token_id,
                early_stopping=False
            )

        # Extract and clean response
        full_response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = full_response[len(prompt):].strip()

        # Clean response
        response = self._clean_response(response)

        # Enhanced validation including allergy safety
        is_personalized = self._validate_personalization(response, current_week)
        is_safe = self._validate_allergy_safety(response, current_week) if has_preferences else True

        print(f"✅ Generated response ({len(response)} chars)")
        print(f"🔍 Personalized: {is_personalized}, Allergy-safe: {is_safe}")

        if len(response) < 50:
            print("⚠️ Response too short")
            return f"Generated response was too short: '{response}'"

        if has_preferences and not is_safe:
            print("⚠️ Response may not be allergy-safe")
            response += "\n\n⚠️ SAFETY REMINDER: Always check ingredient labels and consult healthcare providers about dietary recommendations if you have allergies."

        return response

    except Exception as e:
        print(f"❌ Generation error: {str(e)}")
        import traceback
        traceback.print_exc()
        return f"Generation failed: {str(e)}"

def _validate_allergy_safety(self, response: str, current_week: Dict) -> bool:
    """Basic check for allergy safety in recommendations"""
    preferences = current_week.get('preferences', {})
    allergies = preferences.get('allergies', [])

    if not allergies:
        return True

    response_lower = response.lower()

    # Simple keyword checking (basic safety check)
    risky_words = {
        'Nuts (Tree nuts)': ['nuts', 'almond', 'walnut', 'cashew', 'pecan'],
        'Peanuts': ['peanut', 'peanut butter'],
        'Shellfish': ['shrimp', 'crab', 'lobster', 'shellfish'],
        'Fish': ['salmon', 'tuna', 'fish'],
        'Milk/Dairy': ['milk', 'cheese', 'yogurt', 'dairy'],
        'Eggs': ['eggs', 'egg'],
        'Soy': ['soy', 'tofu', 'soybean'],
        'Wheat/Gluten': ['wheat', 'gluten', 'bread', 'pasta'],
        'Sesame': ['sesame', 'tahini']
    }

    for allergy in allergies:
        if allergy in risky_words:
            for word in risky_words[allergy]:
                if word in response_lower:
                    print(f"⚠️ Potential allergy risk: Found '{word}' but user allergic to {allergy}")
                    return False

    return True

# Well being system

In [ ]:
class WellbeingSystem:
    """Main system that combines data management and LLM"""

    def __init__(self, base_model_id: str, adapter_path: str):
        self.data_manager = SimpleDataManager()
        self.llm = WellbeingLLM(base_model_id, adapter_path)
        print("🎯 WellbeingSystem initialized")

    def analyze_and_recommend(self, user_id: str, week_data: Dict) -> str:
        """Main function: analyze weekly data and generate recommendations"""

        print(f"📊 Analyzing data for user: {user_id}")

        try:
            # Get user history and last recommendation
            user_history = self.data_manager.get_user_history(user_id)
            last_recommendation = self.data_manager.get_last_recommendation(user_id)

            print(f"📈 Found {len(user_history)} weeks of history")
            if last_recommendation:
                print(f"📝 Previous recommendation found ({len(last_recommendation)} chars)")

            # Process current week data
            processed_week = self._process_week_data(week_data)

            # Show what data is being sent to LLM
            print(f"📊 Processed data: {processed_week['total_steps']:,} steps, {processed_week['zone_minutes']} zone mins, {processed_week['avg_sleep']:.1f}h sleep")

            # Generate recommendation using LLM
            recommendation = self.llm.generate_recommendation(
                processed_week, user_history, last_recommendation
            )

            # Save data (with error handling)
            try:
                self.data_manager.save_user_week(user_id, processed_week)
                self.data_manager.save_recommendation(user_id, processed_week['week_start'], recommendation)
                print(f"💾 Data saved successfully")
            except Exception as save_error:
                print(f"⚠️ Warning: Could not save data: {save_error}")
                # Continue anyway - return the recommendation even if saving fails

            print(f"✅ Generated recommendation for {user_id} ({len(recommendation)} chars)")
            return recommendation

        except Exception as e:
            print(f"❌ Error in analyze_and_recommend: {str(e)}")
            import traceback
            traceback.print_exc()
            return f"Analysis failed: {str(e)}. Please check your input data."

    def _process_week_data(self, week_data: Dict) -> Dict:
        """Process and standardize weekly data with better validation"""

        try:
            # Parse sleep and mood data with validation
            sleep_hours = week_data.get('sleep_hours', [])
            mood_scores = week_data.get('mood_scores', [])

            # Handle string input (comma-separated values)
            if isinstance(sleep_hours, str):
                try:
                    sleep_hours = [float(x.strip()) for x in sleep_hours.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid sleep hours format, using default")
                    sleep_hours = []

            if isinstance(mood_scores, str):
                try:
                    mood_scores = [float(x.strip()) for x in mood_scores.split(',') if x.strip()]
                except ValueError:
                    print("⚠️ Invalid mood scores format, using default")
                    mood_scores = []

            # Calculate averages with validation
            avg_sleep = sum(sleep_hours) / len(sleep_hours) if sleep_hours else 0
            avg_mood = sum(mood_scores) / len(mood_scores) if mood_scores else 0

            # Enhanced food data processing
            food_data = self._process_food_data(week_data)

            processed_data = {
                'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                'total_steps': max(0, week_data.get('total_steps', 0)),
                'zone_minutes': max(0, week_data.get('zone_minutes', 0)),
                'exercise_sessions': max(0, week_data.get('exercise_sessions', 0)),
                'avg_sleep': round(avg_sleep, 1),  # <- Make sure this is added
                'avg_mood': round(avg_mood, 1),   # <- Make sure this is added
                'food_data': food_data,
                'timestamp': datetime.now().isoformat()
            }

            return processed_data

        except Exception as e:
            print(f"❌ Error processing week data: {str(e)}")
            # Return empty or default data structure on error
            return {
                 'week_start': week_data.get('week_start', datetime.now().strftime('%Y-%m-%d')),
                 'total_steps': 0,
                 'zone_minutes': 0,
                 'exercise_sessions': 0,
                 'avg_sleep': 0.0,
                 'avg_mood': 0.0,
                 'food_data': {},
                 'timestamp': datetime.now().isoformat(),
                 'processing_error': str(e)
            }


    def _process_food_data(self, week_data: Dict) -> Dict:
        """Process food data handling both old and new formats"""

        food_data = {}

        # Check for new format first (with units)
        unit_fields = ['dairy_liters', 'water_liters', 'legumes_grams', 'meat_grams',
                      'fruits_grams', 'vegetables_grams', 'grains_grams', 'nuts_seeds_grams']

        has_unit_format = any(field in week_data for field in unit_fields)

        if has_unit_format:
            print("📊 Using new format (with units)")
            # New format with units
            for field in unit_fields:
                value = week_data.get(field, 0)
                try:
                    food_data[field] = max(0, float(value))  # Ensure non-negative
                except (ValueError, TypeError):
                    food_data[field] = 0
                    print(f"⚠️ Invalid {field} value, using 0")
        else:
            print("📊 Using old format (servings)")
            # Old format (for backward compatibility)
            old_fields = ['dairy', 'legumes', 'meat', 'fruits', 'vegetables', 'grains', 'nuts_seeds', 'water_glasses']
            for field in old_fields:
                if field in week_data:
                    try:
                        food_data[field] = max(0, float(week_data[field]))
                    except (ValueError, TypeError):
                        food_data[field] = 0

        return food_data

    def get_user_progress(self, user_id: str) -> Dict:
        """Get user's progress over time with enhanced analysis"""

        try:
            history = self.data_manager.get_user_history(user_id)

            if len(history) < 1:
                return {"message": "No data available for this user"}

            if len(history) < 2:
                return {
                    "message": "Need at least 2 weeks of data to show progress",
                    "weeks_tracked": len(history),
                    "latest_week": history[0] if history else None
                }

            # Compare latest vs previous week
            current = history[-1]
            previous = history[-2]

            # Calculate changes
            progress = {
                'weeks_tracked': len(history),
                'steps_change': current['total_steps'] - previous['total_steps'],
                'zone_change': current['zone_minutes'] - previous['zone_minutes'],
                'sleep_change': round(current['avg_sleep'] - previous['avg_sleep'], 1),
                'mood_change': round(current['avg_mood'] - previous['avg_mood'], 1),
                'latest_week': current,
                'previous_week': previous
            }

            # Add trend analysis for longer history
            if len(history) >= 4:
                # Calculate 4-week averages for trends
                recent_4_weeks = history[-4:]
                avg_steps_recent = sum(w['total_steps'] for w in recent_4_weeks) / 4
                avg_sleep_recent = sum(w['avg_sleep'] for w in recent_4_weeks) / 4

                progress['trends'] = {
                    'avg_steps_4weeks': round(avg_steps_recent, 0),
                    'avg_sleep_4weeks': round(avg_sleep_recent, 1),
                    'consistency_score': self._calculate_consistency_score(recent_4_weeks)
                }

            return progress

        except Exception as e:
            print(f"❌ Error getting user progress: {str(e)}")
            return {"error": f"Could not retrieve progress: {str(e)}"}

    def _calculate_consistency_score(self, weeks: List[Dict]) -> float:
        """Calculate a consistency score based on week-to-week variation"""
        if len(weeks) < 2:
            return 0.0

        try:
            # Calculate coefficient of variation for steps (lower = more consistent)
            steps = [w['total_steps'] for w in weeks]
            if all(s == 0 for s in steps):
                return 0.0

            mean_steps = sum(steps) / len(steps)
            if mean_steps == 0:
                return 0.0

            variance = sum((s - mean_steps) ** 2 for s in steps) / len(steps)
            cv = (variance ** 0.5) / mean_steps

            # Convert to score (0-10, where 10 = most consistent)
            consistency_score = max(0, 10 - (cv * 10))
            return round(consistency_score, 1)

        except Exception:
            return 0.0

    def test_system(self) -> Dict:
        """Test the entire system with sample data"""
        print("🧪 Testing WellbeingSystem...")

        test_data = {
            'user_id': 'test_user',
            'week_start': '2024-01-15',
            'total_steps': 55000,
            'zone_minutes': 120,
            'exercise_sessions': 3,
            'sleep_hours': '7.2,6.8,7.5,7.0,6.5,8.0,7.1',
            'mood_scores': '7,6,8,7,6,8,7',
            'dairy_liters': 3.5,
            'water_liters': 14.0,
            'fruits_grams': 1800,
            'vegetables_grams': 3200
        }

        try:
            # Test LLM
            llm_test = self.llm.test_model()
            print(f"LLM Test: {llm_test}")

            # Test full analysis
            recommendation = self.analyze_and_recommend('test_user', test_data)

            # Test progress
            progress = self.get_user_progress('test_user')

            return {
                'llm_test': llm_test,
                'recommendation_length': len(recommendation),
                'recommendation_preview': recommendation[:200] + '...',
                'progress': progress,
                'status': 'success'
            }

        except Exception as e:
            return {
                'error': str(e),
                'status': 'failed'
            }

# Well being app

In [ ]:
# ========================================================================
# COMPLETE WELLBEING APP - FINAL VERSION
# ========================================================================

import gradio as gr
import json
import os
from datetime import datetime
from typing import Dict, List

class WellbeingApp:
    """Complete Gradio interface with targets, history, and preferences"""

    def __init__(self, base_model_id: str, adapter_path: str):
        print("🚀 Initializing Complete WellbeingApp...")
        self.wellbeing_system = WellbeingSystem(base_model_id, adapter_path)

        # Default targets (can be customized per user)
        self.default_targets = {
            'weekly_steps': 70000,
            'weekly_zone_minutes': 150,
            'daily_sleep_hours': 8.0,
            'weekly_water_liters': 14.0,
            'weekly_fruits_grams': 2100,
            'weekly_vegetables_grams': 3500,
            'weekly_exercise_sessions': 4
        }
        print("✅ Complete WellbeingApp ready!")

    def create_wellbeing_app(self):
        """Create comprehensive Gradio interface with tabs"""

        with gr.Blocks(title="🏥 Complete Wellbeing LLM", theme=gr.themes.Soft()) as demo:

            gr.Markdown("# 🏥 Complete AI Wellbeing System")
            gr.Markdown("### Personalized health recommendations with custom targets and progress tracking")

            # Create tabs for different sections
            with gr.Tabs():

                # TAB 1: MAIN ANALYSIS
                with gr.TabItem("🤖 Get Recommendations", elem_id="main-tab"):
                    self._create_main_analysis_tab()

                # TAB 2: TARGETS CUSTOMIZATION
                with gr.TabItem("🎯 Customize Targets", elem_id="targets-tab"):
                    self._create_targets_tab()

                # TAB 3: HISTORY REVIEW
                with gr.TabItem("📈 Progress History", elem_id="history-tab"):
                    self._create_history_tab()

                # TAB 4: SYSTEM INFO
                with gr.TabItem("ℹ️ System Info", elem_id="info-tab"):
                    self._create_info_tab()

        return demo

    def _create_main_analysis_tab(self):
        """Create the main analysis tab"""

        with gr.Row():
            with gr.Column():
                # User Info Section
                gr.Markdown("### 👤 User Information")
                user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Unique identifier for tracking your progress")
                week_start = gr.Textbox(label="📅 Week Start Date", value="2024-01-15", info="Format: YYYY-MM-DD")

                # Preferences Section
                gr.Markdown("### 🍽️ Meal Preferences & Dietary Restrictions")
                with gr.Row():
                    diet_type = gr.Dropdown(
                        label="🥗 Diet Type",
                        choices=[
                            "No Preference", "Vegetarian", "Vegan", "Pescatarian",
                            "Keto", "Paleo", "Mediterranean", "Low Carb", "Gluten Free", "Dairy Free"
                        ],
                        value="No Preference",
                        info="Select your primary dietary preference"
                    )

                    cuisine_preferences = gr.CheckboxGroup(
                        label="🌍 Cuisine Preferences",
                        choices=[
                            "Italian", "Asian", "Mexican", "Indian",
                            "Mediterranean", "American", "Middle Eastern",
                            "Thai", "Japanese", "Greek"
                        ],
                        info="Select cuisines you enjoy (optional)"
                    )

                allergies = gr.CheckboxGroup(
                    label="⚠️ Food Allergies & Intolerances",
                    choices=[
                        "Nuts (Tree nuts)", "Peanuts", "Shellfish", "Fish",
                        "Milk/Dairy", "Eggs", "Soy", "Wheat/Gluten",
                        "Sesame", "Lactose Intolerant", "Other"
                    ],
                    info="⚠️ IMPORTANT: Select all allergies and intolerances"
                )

                other_allergies = gr.Textbox(
                    label="🚨 Other Allergies/Restrictions",
                    placeholder="e.g., tomatoes, specific medications, religious restrictions...",
                    info="Specify any other dietary restrictions or allergies"
                )

                meal_preferences = gr.CheckboxGroup(
                    label="🍴 Meal Preferences",
                    choices=[
                        "Quick meals (under 30 min)", "Meal prep friendly",
                        "High protein focus", "Low sodium", "Heart healthy",
                        "Weight management", "Athletic performance", "Family friendly",
                        "Budget conscious", "Gourmet/complex recipes"
                    ],
                    info="Select preferences that matter to you"
                )

                # Activity Section
                gr.Markdown("### 🏃‍♂️ Physical Activity")
                total_steps = gr.Number(label="Total Steps (Week)", value=65000, info="Your current target will be shown")
                zone_minutes = gr.Number(label="Zone Minutes (Week)", value=120, info="Your current target will be shown")
                exercise_sessions = gr.Number(label="Exercise Sessions", value=3, info="Your current target will be shown")

                # Sleep & Mood Section
                gr.Markdown("### 😴 Sleep & Mood")
                sleep_hours = gr.Textbox(
                    label="Sleep Hours (7 days)",
                    value="7.5,6.8,8.2,7.0,6.5,8.5,7.8",
                    info="Enter daily sleep hours separated by commas"
                )
                mood_scores = gr.Textbox(
                    label="Mood Scores (7 days)",
                    value="7,6,8,7,5,8,7",
                    info="Rate your daily mood from 1-10, separated by commas"
                )

                # Food Section
                gr.Markdown("### 🍎 Weekly Nutrition Intake")
                gr.Markdown("*Enter your total consumption for the entire week*")

                with gr.Row():
                    dairy = gr.Number(label="🥛 Dairy (liters)", value=3.5, info="Milk, yogurt, cheese")
                    water_liters = gr.Number(label="💧 Water (liters)", value=14, info="Your current target will be shown")

                with gr.Row():
                    fruits = gr.Number(label="🍎 Fruits (grams)", value=2100, info="Your current target will be shown")
                    vegetables = gr.Number(label="🥬 Vegetables (grams)", value=3500, info="Your current target will be shown")

                with gr.Row():
                    legumes = gr.Number(label="🫘 Legumes (grams)", value=350, info="Beans, lentils, peas")
                    nuts_seeds = gr.Number(label="🥜 Nuts/Seeds (grams)", value=210, info="Almonds, walnuts, chia seeds")

                with gr.Row():
                    meat = gr.Number(label="🍗 Meat (grams)", value=700, info="Chicken, beef, fish")
                    grains = gr.Number(label="🌾 Grains (grams)", value=2800, info="Rice, bread, pasta, oats")

                # Action buttons
                with gr.Row():
                    analyze_btn = gr.Button("🤖 Get AI Recommendations", variant="primary", size="lg")
                    load_targets_btn = gr.Button("📊 Load My Targets", variant="secondary")

            with gr.Column():
                main_output = gr.Markdown("""
## 📋 How to Use

1. **Enter your User ID** - This tracks your progress over time
2. **Set your dietary preferences** - Diet type, allergies, meal preferences
3. **⚠️ CRITICAL: Select all allergies** - Ensures safe recommendations
4. **Fill in your weekly data** - Activity, sleep, mood, nutrition
5. **Click 'Get AI Recommendations'** - Receive personalized advice

### 🎯 Default Weekly Targets:
- **Steps:** 70,000+ (10,000 daily)
- **Zone Minutes:** 150+ (cardio exercise)
- **Sleep:** 8 hours nightly
- **Water:** 14+ liters (2L daily)
- **Fruits:** 2100+ grams
- **Vegetables:** 3500+ grams
- **Exercise Sessions:** 4+

*Go to 'Customize Targets' tab to modify these targets for your personal goals.*

### 🔒 Privacy & Safety:
- All data stored locally on your device
- AI recommendations respect your dietary restrictions
- Always consult healthcare providers for medical advice
""")

        # Store components for easy access
        self.main_components = {
            'user_id': user_id, 'week_start': week_start, 'total_steps': total_steps,
            'zone_minutes': zone_minutes, 'exercise_sessions': exercise_sessions,
            'sleep_hours': sleep_hours, 'mood_scores': mood_scores,
            'dairy_liters': dairy, 'legumes_grams': legumes, 'meat_grams': meat,
            'fruits_grams': fruits, 'vegetables_grams': vegetables,
            'grains_grams': grains, 'nuts_seeds_grams': nuts_seeds,
            'water_liters': water_liters, 'diet_type': diet_type,
            'cuisine_preferences': cuisine_preferences, 'allergies': allergies,
            'other_allergies': other_allergies, 'meal_preferences': meal_preferences,
            'output': main_output
        }

        # Connect buttons
        analyze_btn.click(
            fn=self.analyze_wellbeing_with_preferences,
            inputs=list(self.main_components.values())[:-1],  # All except output
            outputs=[main_output]
        )

        load_targets_btn.click(
            fn=self.load_user_targets,
            inputs=[user_id],
            outputs=[main_output]
        )


    def _create_targets_tab(self):
        """Create the targets customization tab"""

        gr.Markdown("## 🎯 Customize Your Personal Health Targets")
        gr.Markdown("### Set personalized goals based on your fitness level, health conditions, and personal objectives")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                target_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter the user ID to customize targets for")

                gr.Markdown("### 🏃‍♂️ Activity Targets")
                target_weekly_steps = gr.Number(
                    label="Weekly Steps Target",
                    value=70000,
                    info="Default: 70,000 (10,000 daily). Adjust based on fitness level."
                )
                target_zone_minutes = gr.Number(
                    label="Weekly Zone Minutes Target",
                    value=150,
                    info="Default: 150 minutes. WHO recommends 150+ minutes moderate exercise."
                )
                target_exercise_sessions = gr.Number(
                    label="Weekly Exercise Sessions Target",
                    value=4,
                    info="Default: 4 sessions. Include both cardio and strength training."
                )

                gr.Markdown("### 😴 Sleep & Recovery Targets")
                target_daily_sleep = gr.Number(
                    label="Daily Sleep Hours Target",
                    value=8.0,
                    info="Default: 8 hours. Range: 7-9 hours for most adults."
                )

                gr.Markdown("### 🍎 Nutrition Targets")
                target_weekly_water = gr.Number(
                    label="Weekly Water Target (liters)",
                    value=14.0,
                    info="Default: 14L (2L daily). Adjust for climate and activity level."
                )
                target_weekly_fruits = gr.Number(
                    label="Weekly Fruits Target (grams)",
                    value=2100,
                    info="Default: 2100g (300g daily). 5-a-day recommendation."
                )
                target_weekly_vegetables = gr.Number(
                    label="Weekly Vegetables Target (grams)",
                    value=3500,
                    info="Default: 3500g (500g daily). More vegetables = better health."
                )

                with gr.Row():
                    save_targets_btn = gr.Button("💾 Save My Targets", variant="primary", size="lg")
                    load_current_targets_btn = gr.Button("📊 Load Current Targets", variant="secondary")
                    reset_defaults_btn = gr.Button("🔄 Reset to Defaults", variant="secondary")

            with gr.Column():
                targets_output = gr.Markdown("""
## 🎯 Target Setting Guide

### 📊 Activity Targets:
- **Beginner:** 50,000 steps/week, 100 zone minutes
- **Intermediate:** 70,000 steps/week, 150 zone minutes
- **Advanced:** 90,000+ steps/week, 200+ zone minutes
- **Athletic:** 100,000+ steps/week, 300+ zone minutes

### 😴 Sleep Targets:
- **Age 18-64:** 7-9 hours (recommended 8)
- **Age 65+:** 7-8 hours
- **Athletes:** 8-10 hours for recovery

### 🍎 Nutrition Guidelines:
- **Water:** 2-3L daily (more in hot weather/exercise)
- **Fruits:** 2-4 servings daily (200-400g)
- **Vegetables:** 3-5 servings daily (400-600g)

### 🏥 Medical Considerations:
- **Heart conditions:** Lower intensity targets
- **Diabetes:** Focus on consistent activity
- **Obesity:** Gradual progression important
- **Pregnancy:** Consult healthcare provider

### 💡 Tips for Setting Targets:
1. **Start realistic** - Better to exceed low targets than fail high ones
2. **Progress gradually** - Increase by 10% weekly
3. **Listen to your body** - Adjust based on how you feel
4. **Consider lifestyle** - Work schedule, family commitments
5. **Review monthly** - Adjust targets as you improve

*Remember: These are personal goals. Always consult healthcare providers for medical advice.*
""")

        # Store target components
        self.target_components = {
            'user_id': target_user_id,
            'weekly_steps': target_weekly_steps,
            'zone_minutes': target_zone_minutes,
            'exercise_sessions': target_exercise_sessions,
            'daily_sleep': target_daily_sleep,
            'weekly_water': target_weekly_water,
            'weekly_fruits': target_weekly_fruits,
            'weekly_vegetables': target_weekly_vegetables,
            'output': targets_output
        }

        # Connect buttons
        save_targets_btn.click(
            fn=self.save_user_targets,
            inputs=list(self.target_components.values())[:-1],
            outputs=[targets_output]
        )

        load_current_targets_btn.click(
            fn=self.load_user_targets_display,
            inputs=[target_user_id],
            outputs=list(self.target_components.values()) # Update all target number boxes and output
        )

        reset_defaults_btn.click(
            fn=self.reset_to_defaults,
            outputs=list(self.target_components.values())
        )


    def _create_history_tab(self):
        """Create the history review tab"""

        gr.Markdown("## 📈 Progress History & Analytics")
        gr.Markdown("### Track your health journey and see improvements over time")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 👤 User Selection")
                history_user_id = gr.Textbox(label="👤 User ID", value="user_001", info="Enter user ID to view history")

                gr.Markdown("### 📊 Analysis Options")
                analysis_type = gr.Radio(
                    label="📈 View Type",
                    choices=[
                        "📊 Complete Progress Summary",
                        "📈 Weekly Trends Analysis",
                        "🎯 Target Achievement Report",
                        "📋 Raw Data Export",
                        "🏆 Achievement Milestones"
                    ],
                    value="📊 Complete Progress Summary",
                    info="Choose what type of analysis you want to see"
                )

                weeks_to_show = gr.Slider(
                    label="📅 Weeks to Include",
                    minimum=1,
                    maximum=12,
                    value=4,
                    step=1,
                    info="How many recent weeks to analyze"
                )

                with gr.Row():
                    view_history_btn = gr.Button("📊 View Progress", variant="primary", size="lg")
                    export_data_btn = gr.Button("💾 Export Data", variant="secondary")
                    delete_history_btn = gr.Button("🗑️ Clear History", variant="stop")

            with gr.Column():
                history_output = gr.Markdown("""
## 📈 Progress Tracking Features

### 📊 Available Reports:

**📊 Complete Progress Summary**
- Week-by-week progress overview
- Key metrics trends
- Target achievement rates
- Overall health score

**📈 Weekly Trends Analysis**
- Detailed trend charts (conceptual)
- Improvement/decline patterns
- Seasonal variations
- Consistency analysis

**🎯 Target Achievement Report**
- Which targets you're meeting
- Areas needing improvement
- Success rate by category
- Recommended target adjustments

**📋 Raw Data Export**
- All your historical data
- CSV-style format for external analysis
- Includes recommendations history
- Backup your progress

**🏆 Achievement Milestones**
- Personal records and breakthroughs
- Streak tracking (consistent weeks)
- Goal completions
- Improvement celebrations

### 🔍 What You'll Learn:

- **Patterns:** Best/worst days of week
- **Trends:** Are you improving over time?
- **Consistency:** How steady are your habits?
- **Correlations:** Sleep vs mood, activity vs energy
- **Success factors:** What works best for you

*Select a user and analysis type above to get started.*
""")

        # Store history components
        self.history_components = {
            'user_id': history_user_id,
            'analysis_type': analysis_type,
            'weeks_to_show': weeks_to_show,
            'output': history_output
        }

        # Connect buttons
        view_history_btn.click(
            fn=self.view_user_history,
            inputs=[history_user_id, analysis_type, weeks_to_show],
            outputs=[history_output]
        )

        export_data_btn.click(
            fn=self.export_user_data,
            inputs=[history_user_id],
            outputs=[history_output]
        )

        delete_history_btn.click(
            fn=self.delete_user_history,
            inputs=[history_user_id],
            outputs=[history_output]
        )


    def _create_info_tab(self):
        """Create the system information tab"""

        gr.Markdown("## ℹ️ System Information & Health")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 🧪 System Testing")
                test_system_btn = gr.Button("🧪 Run System Test", variant="primary")
                check_model_btn = gr.Button("🤖 Check LLM Status", variant="secondary")
                check_storage_btn = gr.Button("💾 Check Data Storage", variant="secondary")

            with gr.Column():
                system_output = gr.Markdown("""
## 🏥 Complete Wellbeing System v2.0

### 🎯 Features:
- **Personalized AI Recommendations** based on your data
- **Dietary Preferences & Allergy Management** for safe advice
- **Custom Target Setting** for personalized goals
- **Progress History Tracking** to see improvements
- **Multi-user Support** for families or teams

### 🔒 Privacy & Security:
- **Local Data Storage** - your data stays on your device
- **No Cloud Sync** - complete privacy control
- **Open Source** - transparent and auditable
- **Medical Disclaimer** - for educational purposes only

### 🤖 AI Model Information:
- **Base Model:** Bio-Medical Llama-3-8B
- **Fine-tuned Adapter:** WellBeing_LLM
- **Specialization:** Health and wellness recommendations
- **Safety Features:** Allergy detection and avoidance

### 📊 Data Management:
- **Storage:** Local JSON files
- **Retention:** Last 8 weeks per user
- **Backup:** Manual export available
- **Privacy:** No external data transmission

### 🏥 Medical Disclaimer:
This system provides educational information only. Always consult qualified healthcare professionals for medical advice, diagnosis, or treatment. The AI recommendations are not a substitute for professional medical care.

### 📞 Support:
- Check system status using buttons on the left
- All data is stored locally in `wellbeing_data/` folder
- For technical issues, check the console output
""")

        # Connect info buttons
        test_system_btn.click(
            fn=self.comprehensive_system_test,
            outputs=[system_output]
        )

        check_model_btn.click(
            fn=self.check_llm_status,
            outputs=[system_output]
        )

        check_storage_btn.click(
            fn=self.check_storage_status,
            outputs=[system_output]
        )


    def analyze_wellbeing_with_preferences(self, user_id, week_start, total_steps, zone_minutes, exercise_sessions,
                                         sleep_hours, mood_scores, dairy_liters, legumes_grams, meat_grams,
                                         fruits_grams, vegetables_grams, grains_grams, nuts_seeds_grams, water_liters,
                                         diet_type, cuisine_preferences, allergies, other_allergies, meal_preferences):
        """Enhanced analysis with custom targets and preferences"""

        try:
            # Input validation
            if not user_id or not user_id.strip():
                return "❌ Error: Please enter a valid User ID"

            if not week_start:
                return "❌ Error: Please enter a week start date"

            # Load user's custom targets
            user_targets = self.get_user_targets(user_id.strip())

            # Prepare enhanced data
            week_data = {
                'user_id': user_id.strip(),
                'week_start': week_start,
                'total_steps': max(0, int(total_steps)) if total_steps else 0,
                'zone_minutes': max(0, int(zone_minutes)) if zone_minutes else 0,
                'exercise_sessions': max(0, int(exercise_sessions)) if exercise_sessions else 0,
                'sleep_hours': sleep_hours,
                'mood_scores': mood_scores,
                'dairy_liters': max(0, float(dairy_liters)) if dairy_liters else 0,
                'legumes_grams': max(0, float(legumes_grams)) if legumes_grams else 0,
                'meat_grams': max(0, float(meat_grams)) if meat_grams else 0,
                'fruits_grams': max(0, float(fruits_grams)) if fruits_grams else 0,
                'vegetables_grams': max(0, float(vegetables_grams)) if vegetables_grams else 0,
                'grains_grams': max(0, float(grains_grams)) if grains_grams else 0,
                'nuts_seeds_grams': max(0, float(nuts_seeds_grams)) if nuts_seeds_grams else 0,
                'water_liters': max(0, float(water_liters)) if water_liters else 0,
                'preferences': {
                    'diet_type': diet_type,
                    'cuisine_preferences': cuisine_preferences if cuisine_preferences else [],
                    'allergies': allergies if allergies else [],
                    'other_allergies': other_allergies.strip() if other_allergies else "",
                    'meal_preferences': meal_preferences if meal_preferences else []
                },
                'targets': user_targets  # Include custom targets
            }

            # Generate recommendation with custom targets
            recommendation = self.wellbeing_system.analyze_and_recommend(user_id.strip(), week_data)

            # Enhanced output with custom targets
            output = self._format_analysis_output_with_targets(week_data, recommendation, user_targets)

            return output

        except Exception as e:
            return f"❌ Error: {str(e)}\n\nPlease check your input format and try again."

    def _format_preferences_display(self, preferences: dict) -> str:
        """Format preferences for nice display"""

        lines = ["### 🍽️ Your Dietary Profile:"]

        # Diet type
        if preferences.get('diet_type', 'No Preference') != "No Preference":
            lines.append(f"- **Diet:** {preferences['diet_type']}")

        # Allergies (most important!)
        if preferences.get('allergies'):
            allergy_list = ", ".join(preferences['allergies'])
            lines.append(f"- **⚠️ ALLERGIES:** {allergy_list}")

        if preferences.get('other_allergies'):
            lines.append(f"- **⚠️ OTHER RESTRICTIONS:** {preferences['other_allergies']}")

        # Cuisine preferences
        if preferences.get('cuisine_preferences'):
            cuisine_list = ", ".join(preferences['cuisine_preferences'])
            lines.append(f"- **Preferred Cuisines:** {cuisine_list}")

        # Meal preferences
        if preferences.get('meal_preferences'):
            meal_prefs = ", ".join(preferences['meal_preferences'])
            lines.append(f"- **Meal Style:** {meal_prefs}")

        if len(lines) == 1:  # Only the header
            lines.append("- No specific dietary preferences set")

        return "\n".join(lines)

    def _format_analysis_output_with_targets(self, week_data, recommendation, user_targets):
      """Format analysis output including custom targets"""

      daily_steps = week_data['total_steps'] / 7
      daily_water = week_data['water_liters'] / 7

      # FIX: Safely get avg_sleep and avg_mood with defaults
      avg_sleep = week_data.get('avg_sleep', 0)
      avg_mood = week_data.get('avg_mood', 0)

      # Compare against custom targets
      targets_comparison = f"""
  ### 🎯 Target Performance:
  - **Steps:** {week_data['total_steps']:,} / {user_targets['weekly_steps']:,} {'✅' if week_data['total_steps'] >= user_targets['weekly_steps'] else '⚠️'} ({((week_data['total_steps'] / user_targets['weekly_steps']) * 100):.0f}%)
  - **Zone Minutes:** {week_data['zone_minutes']} / {user_targets['weekly_zone_minutes']} {'✅' if week_data['zone_minutes'] >= user_targets['weekly_zone_minutes'] else '⚠️'} ({((week_data['zone_minutes'] / user_targets['weekly_zone_minutes']) * 100):.0f}%)
  - **Sleep:** {avg_sleep:.1f}h / {user_targets['daily_sleep_hours']}h {'✅' if avg_sleep >= user_targets['daily_sleep_hours'] else '⚠️'}
  - **Water:** {week_data['water_liters']:.1f}L / {user_targets['weekly_water_liters']}L {'✅' if week_data['water_liters'] >= user_targets['weekly_water_liters'] else '⚠️'}
  - **Fruits:** {week_data['fruits_grams']:.0f}g / {user_targets['weekly_fruits_grams']:.0f}g {'✅' if week_data['fruits_grams'] >= user_targets['weekly_fruits_grams'] else '⚠️'}
  - **Vegetables:** {week_data['vegetables_grams']:.0f}g / {user_targets['weekly_vegetables_grams']:.0f}g {'✅' if week_data['vegetables_grams'] >= user_targets['weekly_vegetables_grams'] else '⚠️'}
  """

      preferences_display = self._format_preferences_display(week_data['preferences'])

      return f"""# 🏥 Personalized Wellbeing Analysis & Recommendations

  ## 👤 User: {week_data['user_id']} | 📅 Week: {week_data['week_start']}

  {preferences_display}

  {targets_comparison}

  ### 📊 Your Weekly Data Summary:
  **🏃‍♂️ Activity:**
  - **Steps:** {week_data['total_steps']:,} ({daily_steps:.0f}/day)
  - **Zone Minutes:** {week_data['zone_minutes']}
  - **Exercise Sessions:** {week_data['exercise_sessions']}

  **😴 Sleep & Mood:**
  - **Sleep:** {week_data['sleep_hours']} hours daily
  - **Mood:** {week_data['mood_scores']}/10 daily scores

  **🍎 Nutrition:**
  - 🥛 **Dairy:** {week_data['dairy_liters']:.1f}L | 💧 **Water:** {week_data['water_liters']:.1f}L ({daily_water:.1f}L/day)
  - 🍎 **Fruits:** {week_data['fruits_grams']:.0f}g | 🥬 **Vegetables:** {week_data['vegetables_grams']:.0f}g
  - 🫘 **Legumes:** {week_data['legumes_grams']:.0f}g | 🍗 **Meat:** {week_data['meat_grams']:.0f}g
  - 🌾 **Grains:** {week_data['grains_grams']:.0f}g | 🥜 **Nuts/Seeds:** {week_data['nuts_seeds_grams']:.0f}g

  ---

  ## 🤖 AI Recommendations
  *Personalized based on your data, preferences, allergies, and custom targets*

  {recommendation}

  ---

  💡 **Next Steps:**
  - Follow recommendations that match your dietary preferences
  - **⚠️ Always verify ingredients** if you have allergies
  - Focus on targets marked with ⚠️ for improvement
  - Go to 'Progress History' tab to track improvements over time
  - Adjust targets in 'Customize Targets' if needed

  ✅ **Analysis Complete!** Your personalized data and recommendations have been saved for progress tracking.
  """

    def load_user_targets(self, user_id):
        """Load user targets and format for display in main tab"""
        try:
            if not user_id or not user_id.strip():
                return "❌ Please enter a valid User ID"

            targets = self.get_user_targets(user_id.strip())

            return f"""# 📊 Your Current Targets Loaded

## 🎯 Targets for {user_id}:

### 🏃‍♂️ Activity Targets:
- **Weekly Steps:** {targets['weekly_steps']:,}
- **Zone Minutes:** {targets['weekly_zone_minutes']}
- **Exercise Sessions:** {targets['weekly_exercise_sessions']}

### 😴 Recovery Target:
- **Daily Sleep:** {targets['daily_sleep_hours']} hours

### 🍎 Nutrition Targets:
- **Weekly Water:** {targets['weekly_water_liters']} liters
- **Weekly Fruits:** {targets['weekly_fruits_grams']:,} grams
- **Weekly Vegetables:** {targets['weekly_vegetables_grams']:,} grams

*These targets will be used to evaluate your progress when you get AI recommendations.*

**💡 Tip:** Go to the 'Customize Targets' tab to modify these values if needed.
"""

        except Exception as e:
            return f"❌ Error loading targets: {str(e)}"

    def save_user_targets(self, user_id, weekly_steps, zone_minutes, exercise_sessions,
                          daily_sleep, weekly_water, weekly_fruits, weekly_vegetables):
        """Saves user's custom targets to a file"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to save targets."

        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        try:
            targets_data = {
                'weekly_steps': int(weekly_steps) if weekly_steps else self.default_targets['weekly_steps'],
                'weekly_zone_minutes': int(zone_minutes) if zone_minutes else self.default_targets['weekly_zone_minutes'],
                'weekly_exercise_sessions': int(exercise_sessions) if exercise_sessions else self.default_targets['weekly_exercise_sessions'],
                'daily_sleep_hours': float(daily_sleep) if daily_sleep else self.default_targets['daily_sleep_hours'],
                'weekly_water_liters': float(weekly_water) if weekly_water else self.default_targets['weekly_water_liters'],
                'weekly_fruits_grams': float(weekly_fruits) if weekly_fruits else self.default_targets['weekly_fruits_grams'],
                'weekly_vegetables_grams': float(weekly_vegetables) if weekly_vegetables else self.default_targets['weekly_vegetables_grams'],
            }

            with open(targets_file, 'w') as f:
                json.dump(targets_data, f, indent=2)

            return f"✅ Targets saved successfully for user '{user_id.strip()}'."

        except Exception as e:
            return f"❌ Error saving targets: {str(e)}"

    def get_user_targets(self, user_id: str) -> Dict:
        """Loads user's custom targets, or returns defaults"""
        targets_file = f"{self.wellbeing_system.data_manager.data_dir}/targets_{user_id.strip()}.json"

        if os.path.exists(targets_file):
            try:
                with open(targets_file, 'r') as f:
                    targets = json.load(f)
                    # Ensure all keys from default are present (for backward compatibility)
                    full_targets = self.default_targets.copy()
                    full_targets.update(targets)
                    return full_targets
            except Exception as e:
                print(f"⚠️ Warning: Could not load targets for user '{user_id.strip()}': {str(e)}. Using defaults.")
                return self.default_targets
        else:
            return self.default_targets

    def load_user_targets_display(self, user_id):
        """Loads user targets and formats for display in the targets tab"""
        if not user_id or not user_id.strip():
             return "❌ Error: Please enter a valid User ID to load targets.", *[gr.Number(value=self.default_targets[key]) for key in self.default_targets] # Return defaults to update UI

        targets = self.get_user_targets(user_id.strip())

        output_message = f"✅ Loaded targets for user '{user_id.strip()}'."

        # Return the message and the target values to update the Gradio Number components
        return (
            output_message,
            targets.get('weekly_steps', self.default_targets['weekly_steps']),
            targets.get('weekly_zone_minutes', self.default_targets['weekly_zone_minutes']),
            targets.get('weekly_exercise_sessions', self.default_targets['weekly_exercise_sessions']),
            targets.get('daily_sleep_hours', self.default_targets['daily_sleep_hours']),
            targets.get('weekly_water_liters', self.default_targets['weekly_water_liters']),
            targets.get('weekly_fruits_grams', self.default_targets['weekly_fruits_grams']),
            targets.get('weekly_vegetables_grams', self.default_targets['weekly_vegetables_grams'])
        )

    def reset_to_defaults(self):
        """Resets the target input fields to default values"""
        return (
            "🎯 Resetting targets to defaults.",
            self.default_targets['weekly_steps'],
            self.default_targets['weekly_zone_minutes'],
            self.default_targets['weekly_exercise_sessions'],
            self.default_targets['daily_sleep_hours'],
            self.default_targets['weekly_water_liters'],
            self.default_targets['weekly_fruits_grams'],
            self.default_targets['weekly_vegetables_grams']
        )

    def view_user_history(self, user_id, analysis_type, weeks_to_show):
        """Displays user history based on selected analysis type"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to view history."

        history = self.wellbeing_system.data_manager.get_user_history(user_id.strip())

        if not history:
            return f"📊 No history found for user '{user_id.strip()}'."

        # Limit history to weeks_to_show
        recent_history = history[-weeks_to_show:]

        output = f"## 📈 History for User: {user_id.strip()}\n"
        output += f"📊 **Analysis Type:** {analysis_type}\n"
        output += f"📅 **Weeks Included:** {len(recent_history)}\n\n"

        if analysis_type == "📊 Complete Progress Summary":
            output += "### Weekly Summary (Most Recent First)\n"
            for week in reversed(recent_history):
                output += f"#### Week Start: {week.get('week_start', 'Unknown')}\n"
                output += f"- Steps: {week.get('total_steps', 0):,}\n"
                output += f"- Zone Minutes: {week.get('zone_minutes', 0)}\n"
                output += f"- Sleep (Avg): {week.get('avg_sleep', 0):.1f}h\n"
                output += f"- Mood (Avg): {week.get('avg_mood', 0):.1f}/10\n"
                output += f"- Exercise Sessions: {week.get('exercise_sessions', 0)}\n"
                # Add a snippet of the recommendation if available
                output += f"- Recommendation Snippet: {week.get('recommendation', 'N/A')[:100]}...\n\n"

        elif analysis_type == "📈 Weekly Trends Analysis":
             # This would ideally involve plotting, but we'll provide text summary
            progress = self.wellbeing_system.get_user_progress(user_id.strip())
            output += "### Trends (Last 2 Weeks vs Before)\n"
            if 'message' in progress:
                 output += progress['message']
            else:
                output += f"- Steps Change (Last Week vs Previous): {progress.get('steps_change', 0):+,}\n"
                output += f"- Zone Minutes Change: {progress.get('zone_change', 0):+,}\n"
                output += f"- Sleep Change: {progress.get('sleep_change', 0):+.1f}h\n"
                output += f"- Mood Change: {progress.get('mood_change', 0):+.1f}/10\n"
                if 'trends' in progress:
                    output += "\n### Longer Term Trends (Last 4 Weeks Avg)\n"
                    output += f"- Avg Steps: {progress['trends'].get('avg_steps_4weeks', 0):,.0f}\n"
                    output += f"- Avg Sleep: {progress['trends'].get('avg_sleep_4weeks', 0):.1f}h\n"
                    output += f"- Consistency Score (Steps): {progress['trends'].get('consistency_score', 0):.1f}/10\n"


        elif analysis_type == "🎯 Target Achievement Report":
            user_targets = self.get_user_targets(user_id.strip())
            output += "### Target Achievement (Based on Latest Week)\n"
            latest_week = recent_history[-1] if recent_history else {}

            if latest_week:
                 output += f"- Weekly Steps: {latest_week.get('total_steps', 0):,} / {user_targets['weekly_steps']:,} ({((latest_week.get('total_steps', 0) / user_targets['weekly_steps']) * 100):.0f}%) {'✅' if latest_week.get('total_steps', 0) >= user_targets['weekly_steps'] else '⚠️'}\n"
                 output += f"- Weekly Zone Minutes: {latest_week.get('zone_minutes', 0)} / {user_targets['weekly_zone_minutes']} ({((latest_week.get('zone_minutes', 0) / user_targets['weekly_zone_minutes']) * 100):.0f}%) {'✅' if latest_week.get('zone_minutes', 0) >= user_targets['weekly_zone_minutes'] else '⚠️'}\n"
                 output += f"- Daily Sleep (Avg): {latest_week.get('avg_sleep', 0):.1f}h / {user_targets['daily_sleep_hours']}h {'✅' if latest_week.get('avg_sleep', 0) >= user_targets['daily_sleep_hours'] else '⚠️'}\n"
                 output += f"- Weekly Water (Avg): {latest_week.get('water_liters', 0):.1f}L / {user_targets['weekly_water_liters']}L {'✅' if latest_week.get('water_liters', 0) >= user_targets['weekly_water_liters'] else '⚠️'}\n" # Assuming water is stored with units in processed data
                 output += f"- Weekly Fruits (Avg): {latest_week.get('fruits_grams', 0):.0f}g / {user_targets['weekly_fruits_grams']:.0f}g {'✅' if latest_week.get('fruits_grams', 0) >= user_targets['weekly_fruits_grams'] else '⚠️'}\n"
                 output += f"- Weekly Vegetables (Avg): {latest_week.get('vegetables_grams', 0):.0f}g / {user_targets['weekly_vegetables_grams']:.0f}g {'✅' if latest_week.get('vegetables_grams', 0) >= user_targets['weekly_vegetables_grams'] else '⚠️'}\n"
                 output += f"- Weekly Exercise Sessions: {latest_week.get('exercise_sessions', 0)} / {user_targets['weekly_exercise_sessions']} {'✅' if latest_week.get('exercise_sessions', 0) >= user_targets['weekly_exercise_sessions'] else '⚠️'}\n"

            else:
                output += "No recent data to compare against targets."


        elif analysis_type == "📋 Raw Data Export":
            output += "### Raw Data (JSON Format)\n"
            output += "```json\n"
            output += json.dumps(recent_history, indent=2)
            output += "\n```\n"

        elif analysis_type == "🏆 Achievement Milestones":
            output += "### Achievements (Conceptual - requires more logic)\n"
            output += "This feature is under development. Here are some potential milestones:\n"
            output += "- First week hitting all targets!\n"
            output += "- Longest streak of 10k+ steps daily.\n"
            output += "- Personal best for zone minutes.\n"
            output += "- Consistent 8 hours of sleep for a month.\n"


        return output

    def export_user_data(self, user_id):
        """Exports all user data to a JSON file"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to export data."

        history = self.wellbeing_system.data_manager.get_user_history(user_id.strip())
        recommendations_dir = f"{self.wellbeing_system.data_manager.data_dir}/recommendations"
        user_rec_files = [f for f in os.listdir(recommendations_dir) if f.startswith(f"{user_id.strip()}_") and f.endswith('.json')]

        all_data = {"history": history, "recommendations": []}

        for rec_file in user_rec_files:
            try:
                with open(f"{recommendations_dir}/{rec_file}", 'r') as f:
                    all_data["recommendations"].append(json.load(f))
            except Exception as e:
                print(f"⚠️ Warning: Could not load recommendation file {rec_file}: {str(e)}")


        export_filename = f"wellbeing_export_{user_id.strip()}_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        export_filepath = f"{self.wellbeing_system.data_manager.data_dir}/{export_filename}"

        try:
            with open(export_filepath, 'w') as f:
                json.dump(all_data, f, indent=2)
            return f"✅ Data exported successfully to `{export_filepath}`."
        except Exception as e:
            return f"❌ Error exporting data: {str(e)}"

    def delete_user_history(self, user_id):
        """Deletes all history and recommendations for a user"""
        if not user_id or not user_id.strip():
            return "❌ Error: Please enter a valid User ID to delete history."

        user_file = f"{self.wellbeing_system.data_manager.data_dir}/users/{user_id.strip()}.json"
        recommendations_dir = f"{self.wellbeing_system.data_manager.data_dir}/recommendations"

        deleted_count = 0
        try:
            # Delete user history file
            if os.path.exists(user_file):
                os.remove(user_file)
                deleted_count += 1

            # Delete recommendation files
            user_rec_files = [f for f in os.listdir(recommendations_dir) if f.startswith(f"{user_id.strip()}_") and f.endswith('.json')]
            for rec_file in user_rec_files:
                try:
                    os.remove(f"{recommendations_dir}/{rec_file}")
                    deleted_count += 1
                except Exception as e:
                    print(f"⚠️ Warning: Could not delete recommendation file {rec_file}: {str(e)}")

            if deleted_count > 0:
                 return f"🗑️ Successfully deleted {deleted_count} files for user '{user_id.strip()}'."
            else:
                 return f"ℹ️ No data found for user '{user_id.strip()}'. Nothing to delete."

        except Exception as e:
            return f"❌ Error deleting history: {str(e)}"


    def comprehensive_system_test(self):
        """Runs a comprehensive test of the system components"""
        output = "## 🧪 Comprehensive System Test Results\n"

        try:
            # Test LLM loading and generation
            output += "### 🤖 LLM Test\n"
            llm_test_result = self.wellbeing_system.llm.test_model()
            output += f"- Status: {llm_test_result}\n\n"

            # Test Data Management (basic save/load cycle)
            output += "### 💾 Data Management Test\n"
            test_user = "test_user_system"
            test_week = {
                'week_start': '2024-07-01',
                'total_steps': 75000,
                'zone_minutes': 160,
                'exercise_sessions': 4,
                'sleep_hours': '8,7.5,8,7,7.5,8,8',
                'mood_scores': '8,7,9,8,7,9,8',
                'dairy_liters': 3.0,
                'water_liters': 15.0,
                'fruits_grams': 2200,
                'vegetables_grams': 3600
            }
            test_rec = "Test recommendation output."

            try:
                self.wellbeing_system.data_manager.save_user_week(test_user, test_week)
                history = self.wellbeing_system.data_manager.get_user_history(test_user)
                output += f"- Save/Load History: {'✅ Success' if len(history) > 0 and history[-1].get('total_steps') == 75000 else '❌ Failed'}\n"

                self.wellbeing_system.data_manager.save_recommendation(test_user, test_week['week_start'], test_rec)
                last_rec = self.wellbeing_system.data_manager.get_last_recommendation(test_user)
                output += f"- Save/Load Recommendation: {'✅ Success' if last_rec == test_rec else '❌ Failed'}\n"

                # Clean up test data
                self.wellbeing_system.data_manager.delete_user_history(test_user) # This is now a method on data_manager
                cleanup_check = self.wellbeing_system.data_manager.get_user_history(test_user)
                output += f"- Cleanup: {'✅ Success' if not cleanup_check else '❌ Failed'}\n"

            except Exception as e:
                output += f"- Data Management Test Failed: {str(e)}\n"
                import traceback
                output += f"```\n{traceback.format_exc()}\n```\n"


            # Test Integration (analyze_and_recommend)
            output += "### 🔗 Integration Test\n"
            try:
                # Use sample data that includes preferences
                integration_test_data = {
                    'user_id': 'integration_user',
                    'week_start': '2024-07-08',
                    'total_steps': 50000,
                    'zone_minutes': 100,
                    'exercise_sessions': 2,
                    'sleep_hours': '6.5,7,6,7.5,7,6.8,7.2',
                    'mood_scores': '6,7,6,7,6,7,6',
                    'dairy_liters': 0.5, # Low dairy
                    'water_liters': 10.0, # Low water
                    'fruits_grams': 1500,
                    'vegetables_grams': 2500,
                    'grains_grams': 2000,
                    'nuts_seeds_grams': 150,
                    'meat_grams': 500,
                    'preferences': {
                         'diet_type': 'Vegetarian',
                         'allergies': ['Nuts (Tree nuts)'],
                         'other_allergies': 'Lactose Intolerant',
                         'cuisine_preferences': ['Indian', 'Mexican'],
                         'meal_preferences': ['Quick meals (under 30 min)']
                    }
                }
                # Use default targets for this test
                integration_test_data['targets'] = self.default_targets.copy()


                integration_recommendation = self.wellbeing_system.analyze_and_recommend('integration_user', integration_test_data)

                output += f"- Analyze & Recommend: {'✅ Success' if len(integration_recommendation) > 100 and 'Vegetarian' in integration_recommendation and 'Nuts' not in integration_recommendation else '❌ Failed'}\n"
                output += f"  - Recommendation Length: {len(integration_recommendation)}\n"
                output += f"  - Recommendation Preview: {integration_recommendation[:150]}...\n"

                # Clean up integration test data
                self.wellbeing_system.data_manager.delete_user_history('integration_user')
                cleanup_check_int = self.wellbeing_system.data_manager.get_user_history('integration_user')
                output += f"- Cleanup: {'✅ Success' if not cleanup_check_int else '❌ Failed'}\n"


            except Exception as e:
                output += f"- Integration Test Failed: {str(e)}\n"
                import traceback
                output += f"```\n{traceback.format_exc()}\n```\n"


            output += "\n**🧪 System Test Complete.** Check the results above."

        except Exception as e:
            output += f"\n❌ An error occurred during the system test: {str(e)}"

        return output


    def check_llm_status(self):
        """Checks the status of the loaded LLM model"""
        if self.wellbeing_system.llm.model and self.wellbeing_system.llm.tokenizer:
            status = f"✅ LLM is loaded and ready.\n"
            status += f"- Base Model: {self.wellbeing_system.llm.model.config._name_or_path}\n"
            status += f"- Adapter: {self.wellbeing_system.llm.model.peft_config['default'].peft_adapter_id}\n"
            status += f"- Device: {self.wellbeing_system.llm.device}\n"
            try:
                 test_output = self.wellbeing_system.llm.test_model()
                 status += f"- Basic Test: {test_output}\n"
            except Exception as e:
                 status += f"- Basic Test Failed: {str(e)}\n"

        elif self.wellbeing_system.llm.model:
             status = "⚠️ LLM model is loaded, but tokenizer is missing.\n"
        elif self.wellbeing_system.llm.tokenizer:
             status = "⚠️ LLM tokenizer is loaded, but model is missing.\n"
        else:
            status = "❌ LLM model and tokenizer failed to load.\n"
            status += "Please check the model loading logs in the console for errors."

        return f"## 🤖 LLM System Status\n{status}"

    def check_storage_status(self):
        """Checks the status of the data storage directory"""
        data_dir = self.wellbeing_system.data_manager.data_dir
        output = f"## 💾 Data Storage Status\n"
        output += f"- Data Directory: `{data_dir}`\n"

        try:
            os.makedirs(data_dir, exist_ok=True)
            output += "- Directory exists or created: ✅\n"

            users_dir = f"{data_dir}/users"
            recs_dir = f"{data_dir}/recommendations"

            os.makedirs(users_dir, exist_ok=True)
            os.makedirs(recs_dir, exist_ok=True)

            output += f"- Users directory (`{users_dir}`) exists or created: ✅\n"
            output += f"- Recommendations directory (`{recs_dir}`) exists or created: ✅\n"

            # Count files
            user_files = [f for f in os.listdir(users_dir) if f.endswith('.json')]
            rec_files = [f for f in os.listdir(recs_dir) if f.endswith('.json')]

            output += f"- Number of user data files: {len(user_files)}\n"
            output += f"- Number of recommendation files: {len(rec_files)}\n"

            # Check if a dummy file can be written
            test_file = f"{data_dir}/test_write_{datetime.now().strftime('%Y%m%d%H%M%S')}.txt"
            try:
                with open(test_file, 'w') as f:
                    f.write("test")
                os.remove(test_file)
                output += "- Write permission: ✅\n"
            except Exception as e:
                output += f"- Write permission: ❌ ({str(e)})\n"

        except Exception as e:
            output += f"❌ An error occurred checking storage: {str(e)}\n"

        return output

# Main execution

In [ ]:
# Uncomment to run:
app = WellbeingApp(base_model_id="ContactDoctor/Bio-Medical-Llama-3-8B", adapter_path="AnjaliNV/WellBeing_LLM")
demo = app.create_wellbeing_app()
demo.launch(share=True)